In [ ]:
!kaggle competitions download -c ventilator-pressure-prediction
from zipfile import ZipFile
import os
for i in os.listdir('C:/Users/mehmu/Desktop/PyProjects/Projects/Unlocked/KaggleComp/VentilatorPressurePrediction'):
    if '.zip' in i:
        print(i)
        with ZipFile(i, 'r') as zipObj:
            zipObj.extractall()
        os.remove(i)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc

df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [2]:
df.drop('u_out',axis=1,inplace=True)
df_test.drop('u_out',axis=1,inplace=True)

In [3]:
df.corr()

,id,breath_id,R,C,time_step,u_in,pressure
id,1.000000,0.999999,0.001853,0.007222,-0.000199,-0.002380,-0.002402
breath_id,0.999999,1.000000,0.001860,0.007222,-0.000213,-0.002378,-0.002394
R,0.001853,0.001860,1.000000,-0.096070,-0.014535,-0.148120,0.015976
C,0.007222,0.007222,-0.096070,1.000000,0.004936,0.151002,-0.036727
time_step,-0.000199,-0.000213,-0.014535,0.004936,1.000000,-0.352276,-0.524829
u_in,-0.002380,-0.002378,-0.148120,0.151002,-0.352276,1.000000,0.308136
pressure,-0.002402,-0.002394,0.015976,-0.036727,-0.524829,0.308136,1.000000


In [4]:
X = df.drop(['id','pressure'], axis=1)
Y = df.pressure

sub = df_test.drop('id',axis=1).copy()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)
X_sub_scaled = scaler.transform(sub)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

In [6]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix


In [ ]:
regressor = {
    "RF": RandomForestRegressor(max_depth=3, random_state=2),
    "SVM": SVC(),
    "SGD": SGDRegressor(),
    "KNN": KNeighborsRegressor(), 
    #"LR": LogisticRegression(max_iter=1000), 
    "DT": DecisionTreeRegressor(),
    "MLP": MLPRegressor(max_iter=1000),
    "XGB": XGBRegressor(),
    "LGBM": LGBMRegressor()
}

results = pd.DataFrame(columns=["Regressor", "rmse", "r2"])
for name, clf in regressor.items():
    model = clf
    clf.fit(X_train,y_train)
    predictions = clf.predict(X_val)
    results = results.append({
        "Regressor": name,
        "rmse": np.sqrt(np.square(y_val - predictions)).mean(),
        "r2": 1 - np.square(y_val - predictions).sum()/np.square(y_val - y_val.mean()).sum(),
    }, ignore_index=True)
    
results

In [6]:
from lightgbm import LGBMRegressor
clf=LGBMRegressor()
clf.fit(X,Y)

LGBMRegressor()

In [7]:
y_pred = clf.predict(X_sub_scaled)
submission = pd.read_csv("sample_submission.csv")
submission["pressure"] = y_pred
submission.to_csv('submission.csv', index=False)

In [8]:
!kaggle competitions submit -c ventilator-pressure-prediction -f submission.csv -m ""

Successfully submitted to Google Brain - Ventilator Pressure Prediction



  0%|          | 0.00/103M [00:00<?, ?B/s]
  0%|          | 8.00k/103M [00:00<26:07, 69.2kB/s]
  0%|          | 232k/103M [00:00<01:54, 946kB/s]  
  1%|          | 560k/103M [00:00<01:00, 1.79MB/s]
  1%|          | 752k/103M [00:00<01:08, 1.57MB/s]
  1%|          | 920k/103M [00:00<01:12, 1.49MB/s]
  1%|          | 1.05M/103M [00:00<01:14, 1.43MB/s]
  1%|          | 1.19M/103M [00:01<01:51, 966kB/s] 
  1%|▏         | 1.48M/103M [00:01<02:20, 759kB/s]
  2%|▏         | 1.69M/103M [00:01<01:53, 940kB/s]
  2%|▏         | 1.81M/103M [00:01<01:59, 893kB/s]
  2%|▏         | 1.93M/103M [00:02<03:02, 584kB/s]
  2%|▏         | 2.05M/103M [00:02<02:40, 663kB/s]
  2%|▏         | 2.14M/103M [00:02<03:52, 457kB/s]
  2%|▏         | 2.26M/103M [00:02<03:12, 551kB/s]
  2%|▏         | 2.38M/103M [00:03<02:43, 649kB/s]
  2%|▏         | 2.49M/103M [00:03<02:21, 748kB/s]
  3%|▎         | 2.61M/103M [00:03<02:05, 841kB/s]
  3%|▎         | 2.73M/103M [00:03<01:54, 923kB/s]
  3%|▎         | 2.84M/103M [00:03